<a href="https://colab.research.google.com/github/shannn1/goodRAG/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/hw4
!pwd

Mounted at /content/drive
/content/drive/MyDrive/hw4
/content/drive/MyDrive/hw4


## Fine-tune

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import wandb
from peft import get_peft_model, LoraConfig, TaskType
ds = load_dataset("lighteval/natural_questions_clean")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/956 [00:00<?, ?B/s]

(…)-00000-of-00009-1d732b076b403684.parquet:   0%|          | 0.00/261M [00:00<?, ?B/s]

(…)-00001-of-00009-04e024ad75b84491.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

(…)-00002-of-00009-4c7a9de49d543ea0.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

...

(…)-00007-of-00009-5908e15d74d01a7d.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

(…)-00008-of-00009-a37db32d25a440df.parquet:   0%|          | 0.00/263M [00:00<?, ?B/s]

(…)-00000-of-00001-1a904bb70f827e8a.parquet:   0%|          | 0.00/95.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/106926 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4289 [00:00<?, ? examples/s]

In [ ]:
df_train = ds["train"]
df_validation = ds["validation"]

In [ ]:
def preprocess_dataset(dataset):
    dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ['document', 'question', 'short_answers']])
    def process_short_answers(example):
        example['short_answers'] = example['short_answers'][0] if isinstance(example['short_answers'], list) and len(example['short_answers']) > 0 else None
        return example

    dataset = dataset.map(process_short_answers)

    dataset = dataset.filter(lambda x: x['short_answers'] is not None)

    return dataset

df_train = preprocess_dataset(df_train)
df_validation = preprocess_dataset(df_validation)

Map:   0%|          | 0/106926 [00:00<?, ? examples/s]

Filter:   0%|          | 0/106926 [00:00<?, ? examples/s]

Map:   0%|          | 0/4289 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4289 [00:00<?, ? examples/s]

In [ ]:
'''
train_dataset = Dataset.from_pandas(df_train)
validation_dataset = Dataset.from_pandas(df_validation)
datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

In [ ]:
datasets = DatasetDict({
    "train": df_train,
    "validation": df_validation
})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 序列到序列任务
    inference_mode=False,
    r=8,  # LoRA rank
    lora_alpha=16,
    lora_dropout=0.1
)
model = get_peft_model(model, lora_config)

model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [ ]:
'''
for name, param in model.named_parameters():
    if "decoder.block.23" not in name and "decoder.block.22" not in name and "decoder.block.21" not in name:  # train only last 2 layers
        param.requires_grad = False

trainable_params = [name for name, param in model.named_parameters() if param.requires_grad]
print("Trainable Parameters:", trainable_params)

Trainable Parameters: ['decoder.block.21.layer.0.SelfAttention.q.weight', 'decoder.block.21.layer.0.SelfAttention.k.weight', 'decoder.block.21.layer.0.SelfAttention.v.weight', 'decoder.block.21.layer.0.SelfAttention.o.weight', 'decoder.block.21.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.EncDecAttention.q.weight', 'decoder.block.21.layer.1.EncDecAttention.k.weight', 'decoder.block.21.layer.1.EncDecAttention.v.weight', 'decoder.block.21.layer.1.EncDecAttention.o.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.21.layer.2.DenseReluDense.wi.weight', 'decoder.block.21.layer.2.DenseReluDense.wo.weight', 'decoder.block.21.layer.2.layer_norm.weight', 'decoder.block.22.layer.0.SelfAttention.q.weight', 'decoder.block.22.layer.0.SelfAttention.k.weight', 'decoder.block.22.layer.0.SelfAttention.v.weight', 'decoder.block.22.layer.0.SelfAttention.o.weight', 'decoder.block.22.layer.0.layer_norm.weight', 'decoder.block.22.layer.1.EncDecAttention.q.weight', 'decoder.block

In [ ]:
'''
def answer_question(context, question):
    inputs = tokenizer(context, question, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index])
    )
    return answer

    last_hidden_state = outputs.last_hidden_state
    token_logits = last_hidden_state.mean(dim=2)
    predicted_token_ids = torch.argmax(token_logits, dim=-1)
    generated_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
def preprocess_function(examples):
    inputs = [f"question: {q}  context: {doc}" for q, doc in zip(examples["question"], examples["document"])]
    targets = examples["short_answers"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/106926 [00:00<?, ? examples/s]

Map:   0%|          | 0/4289 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets.save_to_disk("./tokenized_datasets")

In [ ]:
# training
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",    # eval for every epoch
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="wandb",
    run_name="t5-small-finetune1",
    fp16=True
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-3cd5489788ff>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
wandb.init(project="newsqa-finetuning", name="t5-large-finetuning1")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

2214

In [ ]:
trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

In [ ]:
trainer.train()
model.save_pretrained("./finetuned_t5_2")
tokenizer.save_pretrained("./finetuned_t5_2")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
wandb.finish()

## Inference

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "./finetuned_t5_2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

In [ ]:
def generate_answer(document, question, max_length=50):
    input_text = f"question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=3,
            early_stopping=True
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return answer

In [ ]:
df_validation = df_validation.map(
    lambda example: {"predicted_answer": generate_answer(example["question"])},
    batched=False
)


In [ ]:
df_validation

Dataset({
    features: ['document', 'question', 'short_answers', 'predicted_answer'],
    num_rows: 4289
})

## Upload to hf

In [ ]:
from huggingface_hub import login

In [ ]:
login(token="change to your token")

In [ ]:
import pandas as pd
from datasets import Dataset
hf_dataset = Dataset.from_pandas(df_validation)

In [ ]:
dataset_name = "my-preprocessed-dataset"
tokenized_datasets.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Shannnh/my-preprocessed-dataset/commit/6f700570677146286792ee83194533cd0c36b395', commit_message='Upload dataset', commit_description='', oid='6f700570677146286792ee83194533cd0c36b395', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Shannnh/my-preprocessed-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Shannnh/my-preprocessed-dataset'), pr_revision=None, pr_num=None)

In [ ]:
df_validation.push_to_hub("baseline-dataset-5", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Shannnh/baseline-dataset-5/commit/dc1c5b88a8e0c1d0be00efe6417f594a4e9ca1e2', commit_message='Upload dataset', commit_description='', oid='dc1c5b88a8e0c1d0be00efe6417f594a4e9ca1e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Shannnh/baseline-dataset-5', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Shannnh/baseline-dataset-5'), pr_revision=None, pr_num=None)

## Metrics

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Shannnh/baseline-dataset-5")

README.md:   0%|          | 0.00/422 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/88.1M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/4289 [00:00<?, ? examples/s]

In [ ]:
data = dataset["validation"]
def calculate_f1(predicted, ground_truth):
    pred_tokens = predicted.split()
    gt_tokens = ground_truth.split()
    common = set(pred_tokens) & set(gt_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    return 2 * precision * recall / (precision + recall)

f1_scores = [calculate_f1(pred, gt) for pred, gt in zip(data["predicted_answer"], data["short_answers"])]
average_f1 = sum(f1_scores) / len(f1_scores)
print(f"Average F1 Score: {average_f1:.4f}")

Average F1 Score: 0.2950


In [ ]:
average_f1

0.4901781422950149

In [ ]:
def calculate_exact_match(predicted, ground_truth):
    return int(predicted.strip() == ground_truth.strip())

em_scores = [calculate_exact_match(pred, gt) for pred, gt in zip(data["predicted_answer"], data["short_answers"])]
average_em = sum(em_scores) / len(em_scores)
print(f"Exact Match Score: {average_em:.4f}")

Exact Match Score: 0.2024


In [ ]:
average_em

0.0018652366519002097

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def calculate_q_bleu(predicted, ground_truth, question):
    # Tokenize inputs
    ref_tokens = ground_truth.split()
    hyp_tokens = predicted.split()
    question_tokens = question.split()

    reference = [ref_tokens]
    hypothesis = hyp_tokens
    bleu_score = sentence_bleu(reference, hypothesis)

    stop_words = set(stopwords.words('english'))
    important_ref_tokens = [token for token in ref_tokens if token.lower() not in stop_words]
    important_hyp_tokens = [token for token in hyp_tokens if token.lower() not in stop_words]

    key_match = len(set(important_ref_tokens) & set(important_hyp_tokens)) / max(len(set(important_ref_tokens)), 1)

    question_match = len(set(question_tokens) & set(hyp_tokens)) / max(len(set(question_tokens)), 1)

    q_bleu = 0.7 * bleu_score + 0.2 * key_match + 0.1 * question_match

    return q_bleu

data = dataset["validation"]
q_bleu_scores = [
    calculate_q_bleu(pred, gt, q)
    for pred, gt, q in zip(data["predicted_answer"], data["short_answers"], data["question"])
]
average_q_bleu = sum(q_bleu_scores) / len(q_bleu_scores)
print(f"Average Q-BLEU Score: {average_q_bleu:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

Average Q-BLEU Score: 0.0860


In [ ]:
average_q_bleu

0.027138198089277102